# Big Data Project - Simulated Transactions - 21/22

<strong>Andrea Rettaroli</strong> - 0000977930


### Simulated Transactions - Dataset Analysis

The goal of this notebook is to analyze and extract some useful informations from [kaggle simulated-transactions dataset](https://www.kaggle.com/datasets/conorsully1/simulated-transactions). 

The dataset contains ~22GB of data that represents random transactions.
[Notebook used to generate data](https://github.com/conorosully/medium-articles/blob/master/src/transaction_data_generator.ipynb).

Transactions are generated for 75,000 customers and are classified into 12 expenditure types:

- Groceries
- Clothing
- Housing
- Education
- Health
- Motor/Travel
- Entertainment
- Gambling
- Savings
- Bills and Utilities
- Tax
- Fines

Each transaction is represented by 10 features/columns:

- <strong> CUST_ID</strong>: unique ID for every customer
- <strong> START_DATE</strong>: the month the customer started making transactions
- <strong> END_DATE</strong>: the month the customer stopped making transactions
- <strong> TRANS_ID</strong>: unique ID for every transaction
- <strong> DATE</strong>: the date of the transaction
- <strong> YEAR</strong>: the year of the transaction
- <strong> MONTH</strong>: the month of the transaction
- <strong> DAY</strong>: the day of the transaction
- <strong> EXP_TYPE</strong>: the expenditure type (listed above)
- <strong> AMOUNT</strong>: the amount of the transaction (in dollars $)

# Cluster configuration

Let's start by eneble to connetct to spark ui and setting the proper cluster configuration as saw in labs. 

- 3 executors with 3 cores each (leave 1 for daemons; and there's also the AMP (application process manager))
- 8G of memory per executor (recommended 11G, but it exceeds YARN's default maximum allowed in this EMR cluster)

In [3]:
sc.applicationId
"SPARK UI: Enable forwarding of port 20888 and connect to http://localhost:20888/proxy/" + sc.applicationId + "/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res0: String = application_1675691492352_0005
res2: String = SPARK UI: Enable forwarding of port 20888 and connect to http://localhost:20888/proxy/application_1675691492352_0005/


In [ ]:
%%configure -f
{"executorMemory":"8G", "numExecutors":3, "executorCores":3, "conf": {"spark.dynamicAllocation.enabled": "false"}}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1675691492352_0006,spark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1675691492352_0006,spark,idle,Link,Link,✔


# Dataset Preprocessing 

After an analysis of the dataset I decided to:
- Removing the **DATE** column because it is the aggregation of the colums year, month and day that are more useful for the jobs that i'll do.
- creating some smaller datasets to test the correctness of the functions.

This section ne to be executed only the first time.

In [ ]:
// Name of bucket in s3
val bucketname = "unibo-bd2122-arettaroli"
// Paths of datasets
// S3 path of simulated transactions dataset
val s3_path_dataset = "s3a://"+bucketname+"/exam-dataset/transactions.csv"
// S3 path of cleaned dataset without column "date"
val s3_path_dataset_cleaned = "s3a://"+bucketname+"/exam-dataset/transactions-cleaned.csv"

In [ ]:
// Read data from the simulated transactions dataset, drops the column DATE (_c4) then save to another csv.
spark.
  read.
  csv(s3_path_dataset).
  drop("_c4").
  write.
  csv(s3_path_dataset_cleaned)